### Load Modules

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Check GPU

In [2]:
!nvidia-smi

Sat Nov 16 17:52:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.80                 Driver Version: 546.80       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   58C    P0               9W /  37W |      0MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
tf.__version__

'2.10.0'

In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### Read Text Data

#### First Dataset

In [5]:
data = pd.read_csv('./stories.csv')

In [6]:
text_data = [i for i in data['story']]

#### Second Dataset

In [7]:
with open('./jungleb.txt','r',encoding="utf-8") as f:
    text_data_1 = f.readlines()

In [8]:
len(text_data_1)

5767

In [9]:
text_data_1 = text_data_1[75:]

In [10]:
print(f'Second Data Size : {len(text_data_1)}')

Second Data Size : 5692


### Restructure Dataset

In [11]:
def set_dataset(data,n=128):
    return [data[i:i+n] for i in range(0,len(data),n)]

In [12]:
txt_data = []

In [13]:
for i in text_data:
    txt_data.extend(set_dataset(i))

In [14]:
for j in text_data_1:
    txt_data.extend(set_dataset(j))

In [15]:
print(f'Reformed Data Size : {len(txt_data)}')

Reformed Data Size : 6285


### Split Data 

In [16]:
n = int(len(txt_data)*0.95)

In [17]:
train_data = txt_data[:n]
val_data = txt_data[n:]

In [18]:
print(f'Training DataSize : {len(train_data)} , Validation DataSize : {len(val_data)}')

Training DataSize : 5970 , Validation DataSize : 315


### Transform dataset as tensorflow dataset

In [19]:
Tr_Data = tf.data.Dataset.from_tensor_slices((train_data))

In [20]:
Val_Data = tf.data.Dataset.from_tensor_slices((val_data))

### Convert text to numeric

In [21]:
MAX_TOKENS = 5000
BATCH = 32
vec_layer = tf.keras.layers.TextVectorization(max_tokens=MAX_TOKENS)

In [22]:
vec_layer.adapt(Tr_Data,batch_size=None)

In [23]:
vec_layer.vocabulary_size()

5000

In [24]:
vocabulary = vec_layer.get_vocabulary()

### Prepare batchs for data

In [25]:
BUFFER_SIZE = 256

In [26]:
SEQ_LEN = 256
def prepare_batch(data):
    x = vec_layer(data)
    x = x[:,:(SEQ_LEN)]
    x_tr = x[:,:-1]
    y_tr = x[:,1:]
    return (x_tr,y_tr)

In [27]:
def make_batchs(ds):
    return (
        ds.shuffle(BUFFER_SIZE)
        .batch(BATCH)
        .map(prepare_batch,tf.data.AUTOTUNE)
        .prefetch(buffer_size=tf.data.AUTOTUNE)
    )

In [28]:
tr_batch = make_batchs(Tr_Data)

In [29]:
val_batch = make_batchs(Val_Data)

#### Check

In [30]:
for X,Y in tr_batch.take(1):
    break
print(f'Feature Shape : {X.shape} , Target Shape : {Y.shape}')

Feature Shape : (32, 28) , Target Shape : (32, 28)


In [31]:
print(f'X Data ={X[10][:10]}\nY Data ={Y[10][:10]}')

X Data =[3896   20  602    3   20  298  373    4  216    6]
Y Data =[ 20 602   3  20 298 373   4 216   6   1]


## Transformer

### Positional Encoding

In [32]:
def positional_encoding(lenth,depth):
    depth = depth/2
    pos = np.arange(lenth)[:,np.newaxis] # shape (length,1)
    dep = np.arange(depth)[np.newaxis,:]/depth # shape (1,depth)
    angle_rate = 1/(10000**dep) # shape (1,depth)
    angle_rad = pos*angle_rate # shape (length,depth)
    pos_encode = np.concatenate([np.sin(angle_rad),np.cos(angle_rad)],axis=-1)
    return tf.cast(pos_encode,tf.float32)

In [33]:
pos_encoding = positional_encoding(lenth=2048,depth=128)

In [34]:
pos_encoding.shape

TensorShape([2048, 128])

In [35]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self,vocab_size,d_model):
        super().__init__()
        self.d_model = d_model
        self.emb = tf.keras.layers.Embedding(vocab_size,d_model,mask_zero=True)
        self.pos = positional_encoding(lenth=4096,depth=d_model)
    def compute_mask(self, *args, **kwargs):
        return self.emb.compute_mask(*args, **kwargs)
    def call(self,x):
        lenth = tf.shape(x)[1]
        x = self.emb(x)
        x *= tf.math.sqrt(tf.cast(self.d_model,tf.float32))
        x = x + self.pos[tf.newaxis,:lenth,:]
        return x

In [36]:
X.shape

TensorShape([32, 28])

In [37]:
X_emb = PositionalEmbedding(vocab_size=MAX_TOKENS,d_model=128)(X)

In [38]:
X_emb.shape

TensorShape([32, 28, 128])

#### Attention

In [39]:
class BaseAttention(tf.keras.layers.Layer):
    def __init__(self,**kwargs):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()

class Attention(BaseAttention):
    def call(self,x):
        att_output,att_score= self.mha(query=x,value=x,key=x,
                                       return_attention_scores=True,use_causal_mask=True)
        self.attention_score = att_score
        x = self.add([x,att_output])
        x = self.layernorm(x)
        return x

In [40]:
simple_att = Attention(num_heads=2,key_dim=512)

In [41]:
simple_att(X_emb).shape

TensorShape([32, 28, 128])

### FeedForward Network

In [42]:
class FeedForwardNetwork(tf.keras.layers.Layer):
    def __init__(self,d_model,dff,drop_out=0.1):
        super().__init__()
        self.net = tf.keras.Sequential([
            tf.keras.layers.Dense(dff,activation='relu'),
            tf.keras.layers.Dense(d_model),
            tf.keras.layers.Dropout(drop_out)
        ])
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()
    def call(self,x):
        x = self.add([x,self.net(x)])
        x = self.layernorm(x)
        return x

### Create Decoder Layer

In [43]:
class Decoderlayer(tf.keras.layers.Layer):
    def __init__(self,*,d_model,dff,num_heads,dropout_rate=0.1):
        super(Decoderlayer,self).__init__()
        self.attention = Attention(num_heads=num_heads,key_dim=d_model,dropout=dropout_rate)
        self.feed = FeedForwardNetwork(d_model,dff=dff)
    def call(self,x):
        x = self.attention(x)
        self.attention_score = self.attention.attention_score
        x = self.feed(x)
        return x

### Transformer Decoder

In [44]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self,*,num_layers,d_model,dff,num_heads,vocab_size,dropout_rate=0.1):
        super(Decoder,self).__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.num_layers = num_layers
        self.pos_enc = PositionalEmbedding(vocab_size=self.vocab_size,d_model=self.d_model)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.decoder_layers = [
            Decoderlayer(
                d_model=self.d_model,dff=dff,num_heads=num_heads,dropout_rate=dropout_rate
            )
            for _ in range(self.num_layers)
        ]
        self.attention_score = None
    def call(self,x):
        x = self.pos_enc(x)
        x = self.dropout(x)
        for i in range(self.num_layers):
            x = self.decoder_layers[i](x)
        self.attention_score = self.decoder_layers[-1].attention_score
        return x

### Test the Transformer Decoder

In [45]:
sample_decoder = Decoder(num_layers=4,d_model=512,dff=2048,num_heads=8,vocab_size=2000)

In [46]:
output = sample_decoder(X)

In [47]:
output.shape

TensorShape([32, 28, 512])

### Define Transformer

In [48]:
class Transformer(tf.keras.Model):
    def __init__(self,*,num_layers,d_model,dff,num_heads,vocab_size,dropout_rate=0.1):
        super().__init__()
        self.decoder = Decoder(num_layers=num_layers,
                              num_heads=num_heads,
                              d_model=d_model,
                              dff=dff,
                              vocab_size=vocab_size,
                              dropout_rate=dropout_rate)
        self.output_layer = tf.keras.layers.Dense(vocab_size)
    def call(self,inputs):
        x = inputs
        x = self.decoder(x)
        logits = self.output_layer(x)
        try:
            del logits._keras_mask
        except AttributeError:
            pass
        return logits

In [49]:
transformer = Transformer(num_layers=4, 
                          d_model=256, 
                          dff=2048, 
                          num_heads=8, 
                          vocab_size=MAX_TOKENS, 
                          dropout_rate=0.2)

In [50]:
output = transformer(X)

In [51]:
X.shape
output.shape

TensorShape([32, 28, 5000])

In [52]:
transformer.summary()

Model: "transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_1 (Decoder)         multiple                  13901824  
                                                                 
 dense_16 (Dense)            multiple                  1285000   
                                                                 
Total params: 15,186,824
Trainable params: 15,186,824
Non-trainable params: 0
_________________________________________________________________


### Create Custom Scheduler

In [53]:
class CustomScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self,d_model,warmup=4000):
        super().__init__()
        self.d_model = d_model
        self.d_model = tf.cast(self.d_model,tf.float32)
        self.warmup = warmup
    def __call__(self,step):
        step = tf.cast(step,tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup**-1.5)
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [54]:
learning_rate = CustomScheduler(d_model=512)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

### Create loss & Accuracy

In [55]:
def masked_loss(label, pred):
    mask = label != 0
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction="none"
    )
    loss = loss_object(label, pred)

    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask
    loss = tf.reduce_sum(loss) / tf.reduce_sum(mask)
    return loss


def masked_accuracy(label, pred):
    pred = tf.argmax(pred, axis=2)
    label = tf.cast(label, pred.dtype)
    match = label == pred

    mask = label != 0

    match = match & mask

    match = tf.cast(match, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    return tf.reduce_sum(match) / tf.reduce_sum(mask)

### Train

In [56]:
transformer.compile(loss=masked_loss, optimizer=optimizer, metrics=[masked_accuracy])

In [57]:
history = transformer.fit(tr_batch, epochs=500, validation_data=val_batch)

Epoch 1/500
187/187 [==============================] - 12s 46ms/step - loss: 8.0363 - masked_accuracy: 0.0518 - val_loss: 7.0760 - val_masked_accuracy: 0.0632
Epoch 2/500
187/187 [==============================] - 8s 43ms/step - loss: 6.9344 - masked_accuracy: 0.0664 - val_loss: 5.7247 - val_masked_accuracy: 0.0647
Epoch 3/500
187/187 [==============================] - 8s 42ms/step - loss: 6.2174 - masked_accuracy: 0.0898 - val_loss: 5.3955 - val_masked_accuracy: 0.1203
Epoch 4/500
187/187 [==============================] - 8s 43ms/step - loss: 5.9253 - masked_accuracy: 0.1148 - val_loss: 5.1472 - val_masked_accuracy: 0.1899
Epoch 5/500
187/187 [==============================] - 8s 43ms/step - loss: 5.6488 - masked_accuracy: 0.1344 - val_loss: 5.0736 - val_masked_accuracy: 0.1521
Epoch 6/500
187/187 [==============================] - 8s 43ms/step - loss: 5.4001 - masked_accuracy: 0.1484 - val_loss: 5.0015 - val_masked_accuracy: 0.1517
Epoch 7/500
187/187 [==============================

187/187 [==============================] - 11s 61ms/step - loss: 0.5654 - masked_accuracy: 0.8584 - val_loss: 8.1541 - val_masked_accuracy: 0.1059
Epoch 53/500
187/187 [==============================] - 12s 67ms/step - loss: 0.5586 - masked_accuracy: 0.8599 - val_loss: 8.1851 - val_masked_accuracy: 0.1001
Epoch 54/500
187/187 [==============================] - 10s 54ms/step - loss: 0.5459 - masked_accuracy: 0.8634 - val_loss: 8.4208 - val_masked_accuracy: 0.0928
Epoch 55/500
187/187 [==============================] - 12s 65ms/step - loss: 0.5385 - masked_accuracy: 0.8660 - val_loss: 8.6799 - val_masked_accuracy: 0.0861
Epoch 56/500
187/187 [==============================] - 11s 58ms/step - loss: 0.5286 - masked_accuracy: 0.8694 - val_loss: 8.2372 - val_masked_accuracy: 0.1039
Epoch 57/500
187/187 [==============================] - 12s 62ms/step - loss: 0.5231 - masked_accuracy: 0.8691 - val_loss: 8.3698 - val_masked_accuracy: 0.1039
Epoch 58/500
187/187 [==============================]

187/187 [==============================] - 12s 62ms/step - loss: 0.3389 - masked_accuracy: 0.9131 - val_loss: 8.7484 - val_masked_accuracy: 0.1044
Epoch 104/500
187/187 [==============================] - 11s 59ms/step - loss: 0.3355 - masked_accuracy: 0.9151 - val_loss: 8.7309 - val_masked_accuracy: 0.0972
Epoch 105/500
187/187 [==============================] - 12s 63ms/step - loss: 0.3346 - masked_accuracy: 0.9147 - val_loss: 8.8262 - val_masked_accuracy: 0.0968
Epoch 106/500
187/187 [==============================] - 12s 67ms/step - loss: 0.3368 - masked_accuracy: 0.9148 - val_loss: 8.7873 - val_masked_accuracy: 0.1039
Epoch 107/500
187/187 [==============================] - 12s 66ms/step - loss: 0.3305 - masked_accuracy: 0.9161 - val_loss: 8.7592 - val_masked_accuracy: 0.1043
Epoch 108/500
187/187 [==============================] - 11s 57ms/step - loss: 0.3317 - masked_accuracy: 0.9160 - val_loss: 8.7099 - val_masked_accuracy: 0.1013
Epoch 109/500
187/187 [=========================

Epoch 154/500
187/187 [==============================] - 12s 63ms/step - loss: 0.2800 - masked_accuracy: 0.9262 - val_loss: 9.1439 - val_masked_accuracy: 0.0907
Epoch 155/500
187/187 [==============================] - 12s 66ms/step - loss: 0.2789 - masked_accuracy: 0.9272 - val_loss: 8.9196 - val_masked_accuracy: 0.1019
Epoch 156/500
187/187 [==============================] - 13s 70ms/step - loss: 0.2763 - masked_accuracy: 0.9270 - val_loss: 9.1506 - val_masked_accuracy: 0.0924
Epoch 157/500
187/187 [==============================] - 14s 76ms/step - loss: 0.2800 - masked_accuracy: 0.9269 - val_loss: 9.1515 - val_masked_accuracy: 0.0885
Epoch 158/500
187/187 [==============================] - 14s 72ms/step - loss: 0.2762 - masked_accuracy: 0.9277 - val_loss: 8.9393 - val_masked_accuracy: 0.0972
Epoch 159/500
187/187 [==============================] - 13s 71ms/step - loss: 0.2719 - masked_accuracy: 0.9280 - val_loss: 9.0247 - val_masked_accuracy: 0.1001
Epoch 160/500
187/187 [===========

Epoch 205/500
187/187 [==============================] - 25s 136ms/step - loss: 0.2534 - masked_accuracy: 0.9309 - val_loss: 9.2763 - val_masked_accuracy: 0.0898
Epoch 206/500
187/187 [==============================] - 9s 45ms/step - loss: 0.2525 - masked_accuracy: 0.9320 - val_loss: 8.9504 - val_masked_accuracy: 0.1002
Epoch 207/500
187/187 [==============================] - 9s 46ms/step - loss: 0.2501 - masked_accuracy: 0.9318 - val_loss: 9.0549 - val_masked_accuracy: 0.1098
Epoch 208/500
187/187 [==============================] - 22s 118ms/step - loss: 0.2535 - masked_accuracy: 0.9320 - val_loss: 8.9705 - val_masked_accuracy: 0.0905
Epoch 209/500
187/187 [==============================] - 9s 47ms/step - loss: 0.2521 - masked_accuracy: 0.9323 - val_loss: 9.1078 - val_masked_accuracy: 0.0926
Epoch 210/500
187/187 [==============================] - 25s 133ms/step - loss: 0.2543 - masked_accuracy: 0.9321 - val_loss: 9.1107 - val_masked_accuracy: 0.0924
Epoch 211/500
187/187 [===========

Epoch 256/500
187/187 [==============================] - 16s 86ms/step - loss: 0.2360 - masked_accuracy: 0.9340 - val_loss: 9.0420 - val_masked_accuracy: 0.1026
Epoch 257/500
187/187 [==============================] - 13s 65ms/step - loss: 0.2361 - masked_accuracy: 0.9345 - val_loss: 9.4667 - val_masked_accuracy: 0.0863
Epoch 258/500
187/187 [==============================] - 8s 44ms/step - loss: 0.2365 - masked_accuracy: 0.9348 - val_loss: 9.1516 - val_masked_accuracy: 0.0883
Epoch 259/500
187/187 [==============================] - 22s 118ms/step - loss: 0.2384 - masked_accuracy: 0.9350 - val_loss: 9.3697 - val_masked_accuracy: 0.0916
Epoch 260/500
187/187 [==============================] - 8s 44ms/step - loss: 0.2361 - masked_accuracy: 0.9347 - val_loss: 9.0809 - val_masked_accuracy: 0.0893
Epoch 261/500
187/187 [==============================] - 8s 44ms/step - loss: 0.2356 - masked_accuracy: 0.9351 - val_loss: 9.1484 - val_masked_accuracy: 0.0870
Epoch 262/500
187/187 [=============

Epoch 307/500
187/187 [==============================] - 8s 43ms/step - loss: 0.2267 - masked_accuracy: 0.9368 - val_loss: 9.4762 - val_masked_accuracy: 0.0941
Epoch 308/500
187/187 [==============================] - 8s 44ms/step - loss: 0.2271 - masked_accuracy: 0.9371 - val_loss: 9.1953 - val_masked_accuracy: 0.0984
Epoch 309/500
187/187 [==============================] - 21s 112ms/step - loss: 0.2299 - masked_accuracy: 0.9353 - val_loss: 9.2250 - val_masked_accuracy: 0.0933
Epoch 310/500
187/187 [==============================] - 8s 43ms/step - loss: 0.2276 - masked_accuracy: 0.9363 - val_loss: 9.4816 - val_masked_accuracy: 0.0944
Epoch 311/500
187/187 [==============================] - 8s 44ms/step - loss: 0.2309 - masked_accuracy: 0.9357 - val_loss: 9.6436 - val_masked_accuracy: 0.0933
Epoch 312/500
187/187 [==============================] - 21s 115ms/step - loss: 0.2307 - masked_accuracy: 0.9345 - val_loss: 9.1065 - val_masked_accuracy: 0.1052
Epoch 313/500
187/187 [=============

Epoch 358/500
187/187 [==============================] - 18s 96ms/step - loss: 0.2207 - masked_accuracy: 0.9376 - val_loss: 9.3615 - val_masked_accuracy: 0.0978
Epoch 359/500
187/187 [==============================] - 13s 68ms/step - loss: 0.2224 - masked_accuracy: 0.9366 - val_loss: 9.2531 - val_masked_accuracy: 0.1043
Epoch 360/500
187/187 [==============================] - 8s 44ms/step - loss: 0.2205 - masked_accuracy: 0.9380 - val_loss: 9.4326 - val_masked_accuracy: 0.0939
Epoch 361/500
187/187 [==============================] - 24s 128ms/step - loss: 0.2227 - masked_accuracy: 0.9369 - val_loss: 9.4628 - val_masked_accuracy: 0.1019
Epoch 362/500
187/187 [==============================] - 8s 43ms/step - loss: 0.2215 - masked_accuracy: 0.9377 - val_loss: 9.2551 - val_masked_accuracy: 0.0941
Epoch 363/500
187/187 [==============================] - 8s 44ms/step - loss: 0.2229 - masked_accuracy: 0.9368 - val_loss: 9.2865 - val_masked_accuracy: 0.0955
Epoch 364/500
187/187 [=============

Epoch 409/500
187/187 [==============================] - 8s 44ms/step - loss: 0.2169 - masked_accuracy: 0.9382 - val_loss: 9.3671 - val_masked_accuracy: 0.0988
Epoch 410/500
187/187 [==============================] - 8s 44ms/step - loss: 0.2201 - masked_accuracy: 0.9369 - val_loss: 9.2637 - val_masked_accuracy: 0.0966
Epoch 411/500
187/187 [==============================] - 24s 130ms/step - loss: 0.2198 - masked_accuracy: 0.9383 - val_loss: 9.4658 - val_masked_accuracy: 0.0943
Epoch 412/500
187/187 [==============================] - 8s 43ms/step - loss: 0.2175 - masked_accuracy: 0.9382 - val_loss: 9.5239 - val_masked_accuracy: 0.0931
Epoch 413/500
187/187 [==============================] - 8s 44ms/step - loss: 0.2160 - masked_accuracy: 0.9380 - val_loss: 9.6860 - val_masked_accuracy: 0.0823
Epoch 414/500
187/187 [==============================] - 24s 130ms/step - loss: 0.2198 - masked_accuracy: 0.9371 - val_loss: 9.5807 - val_masked_accuracy: 0.0900
Epoch 415/500
187/187 [=============

Epoch 460/500
187/187 [==============================] - 8s 44ms/step - loss: 0.2301 - masked_accuracy: 0.9357 - val_loss: 9.4370 - val_masked_accuracy: 0.0902
Epoch 461/500
187/187 [==============================] - 23s 123ms/step - loss: 0.2314 - masked_accuracy: 0.9350 - val_loss: 9.0108 - val_masked_accuracy: 0.0967
Epoch 462/500
187/187 [==============================] - 9s 45ms/step - loss: 0.2301 - masked_accuracy: 0.9355 - val_loss: 9.1475 - val_masked_accuracy: 0.0966
Epoch 463/500
187/187 [==============================] - 9s 46ms/step - loss: 0.2298 - masked_accuracy: 0.9354 - val_loss: 9.0969 - val_masked_accuracy: 0.1145
Epoch 464/500
187/187 [==============================] - 23s 124ms/step - loss: 0.2326 - masked_accuracy: 0.9346 - val_loss: 9.1435 - val_masked_accuracy: 0.1016
Epoch 465/500
187/187 [==============================] - 9s 49ms/step - loss: 0.2334 - masked_accuracy: 0.9348 - val_loss: 9.1953 - val_masked_accuracy: 0.0990
Epoch 466/500
187/187 [=============

### Inference ( Test The Model)

### Build Generator

In [58]:
class Generator(tf.Module):
    def __init__(self,
                tokenizer,
                transformer,
                vocabulary,
                max_token,
                temp=0.0):
        self.tokenizer = tokenizer
        self.transformer = transformer
        self.vocab = vocabulary
        self.max_token = max_token
        self.temp = temp
    def __call__(self,keyword,max_length=MAX_TOKENS):
        input_sentence = self.tokenizer(keyword)
        input_sentence = tf.expand_dims(input_sentence,axis=0)
        output_array = tf.TensorArray(dtype=tf.int64,size=0,dynamic_size=True)
        print(f'Generating {self.max_token} number of new tokens ------')
        for i in tf.range(self.max_token):
            output = tf.transpose(output_array.stack())
            prediction = self.transformer(input_sentence,training=False)
            prediction = prediction[:,-1:,:]
            if self.temp==0.0:
                prediction_id = tf.argmax(prediction,axis=-1)
            else:
                prediction = prediction/self.temp
                prediction_id = tf.random.categorical(prediction[0],
                                                     num_samples=1)
            output_array = output_array.write(i+1,prediction_id[0])
            input_sentence = tf.experimental.numpy.append(input_sentence,
                                                         prediction_id[0])
            input_sentence = tf.expand_dims(input_sentence,axis=0)
        output = tf.transpose(output_array.stack())
        id_to_word = tf.keras.layers.StringLookup(mask_token="",
                                          vocabulary=vocabulary, 
                                          oov_token="[UNK]", 
                                          invert=True)
        print(f'Using Temparature of {self.temp} -----')
        text = id_to_word(output)
        tokens = output
        self.transformer(output[:,:-1],training=False)
        attention_weights = self.transformer.decoder.attention_score
        
        return text,tokens,attention_weights

### Run

In [59]:
gen_text_size = 100
temp = 0.9
generator = Generator(tokenizer=vec_layer,
                     transformer=transformer,
                     vocabulary=vocabulary,
                     max_token=gen_text_size,
                     temp=temp)

In [60]:
def print_generated_text(input_keyword,gen_text):
    print(f' Input : {input_keyword} ')
    print(f' Generated : {gen_text}')

In [61]:
input_word = "You"
gen_text,gen_token,att_score = generator(input_word)
print_generated_text(input_word,gen_text)

Generating 100 number of new tokens ------
Using Temparature of 0.9 -----
 Input : You 
 Generated : [[b'' b'the' b'instant' b'you' b'feel' b'the' b'rein' b'on' b'your'
  b'neck' b'if' b'you' b'haven\xe2\x80\x99t' b'room' b'to' b'another'
  b'world' b'the' b'bottom' b'of' b'a' b'country' b'the' b'way' b'you'
  b'end' b'of' b'[UNK]' b't' b'oh' b'camel' b'with' b'your' b'find'
  b'out\xe2\x80\x9d' b'me' b'as' b'i' b'could' b'see' b'the' b'moon'
  b'if' b'you' b'had' b'been' b'two' b'tails' b'if' b'you' b'had' b'been'
  b'waked' b'the' b'past' b'his' b'eyes' b'wide' b'with' b'your' b'neck'
  b'it' b'was' b'i' b'had' b'been' b'here' b'is' b'there' b'is' b'a'
  b'leg' b'or' b'afraid' b'of' b'the' b'lying' b'a' b'shot' b'up' b'red'
  b'[UNK]' b'down' b'and' b'i' b'could' b'see' b'the' b'amir' b'away'
  b'from' b'the' b'elephants' b'were' b'afraid' b'of' b'[UNK]' b'just'
  b'because' b'i' b'could']]


In [62]:
print( input_word + ' '.join([i.decode('utf-8') for i in gen_text[0].numpy()]))

You the instant you feel the rein on your neck if you haven’t room to another world the bottom of a country the way you end of [UNK] t oh camel with your find out” me as i could see the moon if you had been two tails if you had been waked the past his eyes wide with your neck it was i had been here is there is a leg or afraid of the lying a shot up red [UNK] down and i could see the amir away from the elephants were afraid of [UNK] just because i could
